In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
import os

# File path for the CSV
csv_file_path = "cix.csv"

# Create the necessary widgets
string_box = widgets.Text(description='Ticker')
description_box = widgets.Text(description='Description')
save_button = widgets.Button(description='Save')
my_cix_button = widgets.Button(description="My Cix's")

output_save = widgets.Output()
output_display = widgets.Output()

# Function to save the entries to the CSV file
def save_to_csv(b):
    with output_save:
        output_save.clear_output()
        string = string_box.value
        description = description_box.value

        # Create a DataFrame from the inputs
        df = pd.DataFrame([[string, description]], columns=['String', 'Description'])

        # Save to CSV
        if os.path.exists(csv_file_path):
            df.to_csv(csv_file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(csv_file_path, index=False)

        #print(f'Saved: {string}, {description}')

# Function to display the contents of the CSV file
def display_cix(b):
    with output_display:
        output_display.clear_output()
        if os.path.exists(csv_file_path):
            df = pd.read_csv(csv_file_path)
            # Center the DataFrame display
            styled_table = df.style.set_table_styles([{
                'selector': 'table',
                'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
            }]).to_html()

            centered_html = f'<div class="centered-output"><div class="centered-table">{styled_table}</div></div>'
            display(HTML(centered_html))
        else:
            print('No entries found.')

# Set button click events
save_button.on_click(save_to_csv)
my_cix_button.on_click(display_cix)

# Display widgets
display(string_box, description_box, save_button, output_save, my_cix_button, output_display)

# Load custom CSS for centering the table
display(HTML('''
<style>
.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}
.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))


Text(value='', description='Ticker')

Text(value='', description='Description')

Button(description='Save', style=ButtonStyle())

Output()

Button(description="My Cix's", style=ButtonStyle())

Output()

In [4]:
import pandas as pd
from xbbg import blp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from datetime import timedelta as td
import statsmodels.api as sm
import re
import matplotlib.image as image
from ipywidgets import interact, IntSlider, Checkbox, Dropdown, Output, HBox, VBox, interactive, interactive_output, ToggleButton,Text, Button, DatePicker, IntText, ToggleButtons, RadioButtons,SelectMultiple
from IPython.display import display, clear_output
import itertools
from scipy import stats
from scipy.optimize import minimize 
from scipy.special import ndtr

spot_library = {'eur_6':['EUSA', ' Curncy'],'eur': ['EESWE', ' Curncy'], 'usd': ['USOSFR', ' Curncy'], 
                'gbp': ['BPSWS', ' Curncy'],'chf': ['SFSNT', ' Curncy'],'sek': ['SKSW', ' Curncy'],
                 'nok': ['NKSW', ' Curncy'], 'hkd': ['HDSW', ' Curncy'],'czk': ['CKSW', ' Curncy'],
                 'pln': ['PZSW', ' Curncy'],'ils':['ISSW', ' Curncy'],  'cad':['CDSW', ' Curncy'], 
                 'jpy':['JYSO', ' Curncy'], 'aud': ['ADSW', ' Curncy'],'sgd':['SDSW', ' Curncy'],
                'krw': ['KWSWNI', ' Curncy'],
                'zar': ['SASW', ' Curncy'],
                'nzd': ['NDSW', ' Curncy'],
                'mxn': ['MPSW', ' Curncy']} 

forward_library = {'eur_6': ['EUSA', ' Curncy'], 
                 'eur': ['S0514FS ', ' BLC Curncy'], 
                 'usd': ['S0490FS ', ' BLC Curncy'], 
                 'gbp': ['S0141FS ', ' BLC Curncy'],
                 'chf': ['S0234FS ', ' BLC Curncy'],
                 'sek': ['SD0020FS ', ' BLC Curncy'],
                 'nok': ['SD0313FS ', ' BLC Curncy'],
                 'hkd': ['HDFS', ' Curncy'],
                 'czk': ['S0320FS ', ' BLC Curncy'],
                 'pln': ['S0323FS ', ' BLC Curncy'],
                 'ils': ['ISFS', ' Curncy'],
                 'cad': ['CDFS', ' Curncy'],
                 'jpy': ['S0195FS ', ' BLC Curncy'],
                 'aud': ['SD0302FS ', ' BLC Curncy'],
                'sgd': ['SDFS', ' Curncy'],
                'krw': ['S0205FS ', ' BLC Curncy'],
                'zar': ['SAFS', ' Curncy'],#
                'nzd': ['SD0015FS ', ' BLC Curncy'],
                'mxn': ['SD0083FS ', ' BLC Curncy']} 

################################################# SWAP STRUCTURE BUILDERS ################################################

def cut(some):
    x = 0
    y = some[0]
    while y.isdigit():
        y=some[x]
        x+=1
    return some[:x],some[x:]


def f(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num

def t(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        else:
            num+='0'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num


def spot_ticker(dex,tenor):
    if dex.lower() == 'mxn':
        y = int(tenor[:-1]) * 13
        num = f'{y//12}{chr(64+(y%12))}'
    else:
        num = tenor[:-1]
    return f'{spot_library[dex][0]}{num}{spot_library[dex][1]}'

def forward_ticker(dex,fwd):
    dex = dex.lower()
    fwd = fwd.lower()
    old = ['eur_6','hkd','ils','cad','sgd','zar']
    
    if cut(fwd)[0] == '0y':
        return spot_ticker(dex,cut(fwd)[1])
    elif dex == 'eur_6':
        F,T = f(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    elif dex in old:
        F,T = t(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    else:
        
        return f'{forward_library[dex][0]}{fwd.upper()}{forward_library[dex][1]}'


def swap_structure(dex,structure,start, end = 'today',bps = True):
    f_fly = structure.count('/') == 2
    f_crv = structure.count('/') == 1  
    fly = structure.count('s') == 3
    crv = structure.count('s') == 2
    out = max(2 -sum([i.isalpha() for i in structure]),0)
    
    if f_fly or f_crv:
        legs = [forward_ticker(dex,i) for i in structure.split('/')]
    elif fly or crv:
        legs = [spot_ticker(dex,i) for i in [i+'Y' for i in structure.split('s') if i.isdigit()]]
    else:
        legs = forward_ticker(dex,('0Y'*out) + structure)
        
    df = blp.bdh(legs, flds='px_last', start_date=start,end_date=end).fillna(method ='bfill')  *(100 if bps else 1)
    s  = pd.DataFrame({})
    if f_fly or fly:
        x = (2 * df.iloc[:,1]) - (df.iloc[:,0] + df.iloc[:,2])
    elif f_crv or crv:
        x = df.iloc[:,1] - df.iloc[:,0]
    else:
        x = df.iloc[:,0]
        
    s[f'{dex.upper()} {structure}'] = x
    return s

################################################# BOND STRUCTURE BUILDERS ################################################
def bond_ticker(bond_name, bond_tenor):
    cmb = {'ust':1,'dbr':2,'spgb':8,'frtr':4,'btp':5,'ukt':6}
    return f'RV000{cmb[bond_name]}P {bond_tenor.upper()} BLC Curncy'

def bond_structure(bonds,structure, start, end = 'today'):
    if '-' in bonds:
        s = pd.DataFrame({})
        bond1,bond2 = bonds.split('-')
        s[f'{bonds.upper()} {structure}'] = bond_structure(bond1,structure, start, end)[f'{bond1.upper()} {structure}'] - bond_structure(bond2,structure, start, end)[f'{bond2.upper()} {structure}']
        return s
    else:
        if 'y' in structure.lower(): #bond o/r
            tickers = bond_ticker(bonds.lower(), structure)
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = [f'{bonds.upper()} {structure}']
            return base
        
        elif structure.lower().count('s') == 2: #bond curve
            tickers = [bond_ticker(bonds.lower(), f'{i}y') for i in structure.lower().split('s') if i != '']
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = tickers
            s = pd.DataFrame({})
            s[f'{bonds.upper()} {structure}'] = base[tickers[1]] - base[tickers[0]]
            return s
        
        elif structure.lower().count('s') == 3: # bond fly
            tickers = [bond_ticker(bonds.lower(), f'{i}y') for i in structure.lower().split('s') if i != '']
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = tickers
            s = pd.DataFrame({})
            s[f'{bonds.upper()} {structure}'] = (2*base[tickers[1]]) - (base[tickers[0]] + base[tickers[2]])
            return s

################################################# FUTURES STRUCTURE BUILDERS ################################################

def fut_ticker(dex, contract, name = False):
    x  = ''
    if dex.lower()[:3] == 'eur':
        x = 'ER'
    elif dex.lower() == 'usd':
        x = 'SFR'
    elif dex.lower() == 'gbp':
        x = 'SFI'
    
    if name:
        return x
    else:
        return f'{x}{contract} Comdty'   
           
    
def fut_structure(dex,structure, start, end = 'today'):
    name = fut_ticker(dex.lower(), 0,True)
    
    if structure.lower().count('s') == 2: #fut curve
        tickers = [fut_ticker(dex.lower(), int(i)) for i in structure.lower().split('s') if i != '']
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{name}{structure.replace("s","")}'] = base[tickers[0]] - base[tickers[1]]
        return s
        
    elif structure.lower().count('s') == 3: # fut fly
        tickers = [fut_ticker(dex.lower(), int(i)) for i in structure.lower().split('s') if i != '']
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{name}{structure.replace("s","")}'] = (2*base[tickers[1]]) - (base[tickers[0]] + base[tickers[2]])
        return s
    else:
        tickers = fut_ticker(dex.lower(), structure)
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = [f'{name}{structure}']
        return base

################################################# ASW STRUCTURE BUILDERS ################################################

def get_asw(bond_name,tenor,start, end ='today',euro = 'estr'):
    asso_swap = {'ust':'usd','dbr':'eur','spgb':'eur','frtr':'eur','btp':'eur','ukt':'gbp'}
    
    if asso_swap[bond_name] == 'eur' and euro != 'estr':
        tickers = [bond_ticker(bond_name,tenor), spot_ticker(asso_swap[bond_name]+'_6',tenor)]
    else:
        tickers = [bond_ticker(bond_name,tenor), spot_ticker(asso_swap[bond_name],tenor)]
    base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
    base.columns = tickers
    s = pd.DataFrame({})
    s[f'{tenor.upper()} {bond_name.upper()} ASW'] = base[tickers[1]] - base[tickers[0]]
    return s

def asw_structure(bonds,structure, start, end = 'today',euro = 'estr'):
    if '-' in bonds:
        s = pd.DataFrame({})
        bond1,bond2 = bonds.split('-')
        s[f'{bonds.upper()} ASW {structure}'] = asw_structure(bond1,structure, start, end,euro)[f'{bond1.upper()} ASW {structure}'] - asw_structure(bond2,structure, start, end,euro)[f'{bond2.upper()} ASW {structure}']
        return s
    else:
        if 'y' in structure.lower(): #asw o/r
            return get_asw(bonds,structure,start, end,euro)
        
        elif structure.lower().count('s') == 2: #asw curve
            legs = [ f'{i}y' for i in structure.lower().split('s') if i != '']
            short_leg, long_leg = [get_asw(bonds,i,start,end,euro) for i in legs]
            df = short_leg.join(long_leg, how = 'inner')
            s = pd.DataFrame({})
            s[f'{bonds.upper()} ASW {structure}'] = df.iloc[:,1]  - df.iloc[:,0]  
            return s
        
        elif structure.lower().count('s') == 3: # asw fly
            legs = [ f'{i}y' for i in structure.lower().split('s') if i != '']
            short_leg, belly ,long_leg = [get_asw(bonds,i,start,end,euro) for i in legs]
            short_leg = short_leg.join(belly, how = 'inner')
            df = short_leg.join(long_leg, how = 'inner')
            s = pd.DataFrame({})
            s[f'{bonds.upper()} ASW {structure}'] = (2 * df.iloc[:,1])  - (df.iloc[:,0] + df.iloc[:,2])  
            return s
        

######################################### TICKER HANDLERS #################################################################

def get_other(ticker,attribute,title,start,end='today' ):
    if 'price' in attribute.lower():
        attribute = 'px_last'
    else:
        attribute = 'YLD_YTM_MID'
    base = blp.bdh(ticker,attribute,start,end)
    base.columns = [title]
    return base

######################################### INVOICE SPREAD STRUCTURE BUILDERS ################################################


def get_ivsp(country,start, end ='today',which = 'TUA',euro ='estr'):
    tkr = {'USD':['TUAISPS','3YAISPS','FVAISPS','TYAISPS','UXYAISPS','USAISPS','WNAISPS'],
           'GER':['DUAISP','OEAISP','RXAISP','UBAISP'],
           'ITA':['BTAISP','IKAISP'],
           'FRA':['OATAISP'],
           'GBP':['GAISPO']}
    
    if country.upper() == 'USD' or country.upper() == 'GER' or country.upper() == 'ITA' :
        for i in tkr[country.upper()]:
            if which in i:
                future = i
    else:
        future = tkr[country.upper()][0]
    
    if country.lower() in ['ger','ita','fra'] and euro == 'estr':
        future += 'E'
    final_ticker = f'{future} Comdty'
    base = blp.bdh(final_ticker,'px_last',start,end)
    base.columns = [f'{future} Comdty']    
    return base
                
######################################## MEETING DATES HANDLER ##################################################################
    
def md_swap_ticker(bank, meeting,m_of_m =False):
    central_bank = {'FOMC':['USSOFED',' Curncy'],
                'ECB':['EESF','A Curncy'],
                'MPC':['GPSF','A Curncy']}
    ticker = f'{central_bank[bank.upper()][0]}{meeting}{central_bank[bank.upper()][1]}' 
    if m_of_m:
        return blp.bdp(ticker, 'SW_EFF_DT').values.flatten()[0].strftime('%b')
    else:
        return ticker
    
def get_md_swap(bank,structure,start,end  = 'today'):
    if structure.lower().count('s') == 2:
        tickers = [md_swap_ticker(bank.upper(),int(i)) for i in structure.lower().split('s') if i != ''] 
        lbls = ''.join([md_swap_ticker(bank.upper(),int(i), True) for i in structure.lower().split('s') if i != ''])
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{lbls} {bank.upper()}'] = base[tickers[1]] - base[tickers[0]]
        return s
        
    if structure.lower().count('s') == 3:
        tickers = [md_swap_ticker(bank.upper(),int(i)) for i in structure.lower().split('s') if i != '']
        lbls = ''.join([md_swap_ticker(bank.upper(),int(i), True) for i in structure.lower().split('s') if i != ''])            
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{lbls} {bank.upper()}'] = (2*base[tickers[1]] )- (base[tickers[0]] + base[tickers[2]])
        return s
    else:
        tickers = md_swap_ticker(bank.upper(), structure)
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = [f'{md_swap_ticker(bank.upper(), structure,True)}{bank.upper()}']
        return base

    
################################################# FX HANDLER ############################################################
def get_fx_pair(pair,start,end='today'):
    ticker = f'{pair.upper()} Curncy'
    base = blp.bdh(ticker,'px_last',start,end)
    base.columns = [pair.upper()]
    return base

######################################## BOND FUTURE HANDLER ##################################################################
def get_bondf_ticker(name):
    tic_dic = {'DU':'DUA Comdty','OE':'OEA Comdty','RX': 'RXA Comdty','UB': 'UBA Comdty',
                'BTS': 'BTSA Comdty','IK': 'IKA Comdty',
                'OAT': 'OATA Comdty',
                'G':'G A Comdty',
                'TU': 'TUA Comdty','FV': 'FVA Comdty', 'TY': 'TYA Comdty','WN': 'WNA Comdty','UXY': 'UXYA Comdty'}
    return tic_dic[name.upper()]
    

def get_bond_fut(_str, start,end = 'today', fld = 'yield'):
    fld = 'YLD_YTM_MID' if fld.lower() == 'yield' else 'PX_LAST' 
    _str = _str.upper()
    label = _str.split('/')
    legs = [get_bondf_ticker(i) for i in label]
    bps = 100 if fld == 'YLD_YTM_MID' else 1
    base  = blp.bdh(legs,fld,start,end) * bps
    base.columns = label
    s = pd.DataFrame({})
    
    if _str.count('/') == 2:
        s[_str] = 2 * base[label[1]] - (base[label[0]] + base[label[2]]) 
            
    elif _str.count('/') == 1:
        s[_str] =  base[label[1]] - base[label[0]] 
                                        
    else:
        s = base
        
    return s
    

######################################## INFL SWAP HANDLER ##################################################################



######################################## MASTER HANDLER ##################################################################
def get_cix_options():
    if os.path.exists('cix.csv'):
        df = pd.read_csv('cix.csv')
        return df.iloc[:,0].tolist()
    else:
        return []

def function_for_bb(trades,_from,_to):
    master_c = pd.DataFrame({})
    for i in trades.keys():
        if len(trades[i]) != 0:
            for j in trades[i]:
                if i == 'SWAP':
                    h = swap_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)                                
                elif i == 'FUTURE':
                    h = fut_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)                              
                elif i == 'BOND':
                    h = bond_structure(bonds = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                elif i ==  'ASW':
                    h = asw_structure(bonds = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to, euro = j[2].lower())
                elif i == 'XMKT SWAP':
                    h1 = swap_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                    h2 = swap_structure(dex = j[2].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                    h_int = h1.join(h2, how = 'inner')
                    h = pd.DataFrame(h_int.iloc[:,0] - h_int.iloc[:,1], columns = [f'{j[0]}-{j[2]} {j[1]}'])
                elif i == 'MD SWAP':
                    h = get_md_swap(bank = j[0].upper(),structure=j[1].lower() ,start = _from , end = _to)
                elif i ==  'BOND FUTURE':
                    h = get_bond_fut(_str = j[0],start = _from , end = _to,fld =  j[1])
                elif i == 'FX':
                    h = get_fx_pair(pair = j[0].upper(),start = _from , end = _to)
                elif i == 'OTHER/CIX':
                    h = get_other(ticker = j[0], attribute= j[1], title = j[2], start = _from , end = _to)            
                elif i == 'MY CIX':
                    #print(j[0], j[0], j[0], _from, _to)
                    h = get_other(ticker = j[0], attribute= j[1], title = j[2], start = _from , end = _to)   
                if master_c.shape == (0,0):
                    master_c = h
                else:
                    master_c = master_c.join(h, how = 'inner')
    return master_c

In [5]:
from IPython.display import HTML, display
import ipywidgets as widgets

clrs = ['whites','reds','greens','blues']
def fut_presets(dex,clr):
    color = {key : range(1+(clrs.index(key)*4),5+(clrs.index(key)*4)) for key in clrs}
    fin_clr = []
    for i in clr.lower().split(', '):
        fin_clr += list(color[i])
    for i in fin_clr:
        yield (dex.upper(), str(i))

clr_dd = [', '.join([m.capitalize() for m in j]) for i in range(1,len(clrs)+1) for j in itertools.combinations(clrs, i) ]


sp_or_fw = ['Spot Tenors', 'Forward Gaps']
yc = ['(up to 10y)','(up to 15y)','(up to 30y)','(up to 50y)']
srt = ['Spot Curves', 'Spot Flys', 'Forward Curves']
swap_ops = [f'{i} {j}' for i in sp_or_fw for j in yc] + srt
bond_ops = ['Outright', 'Curves', 'Flys']

def swap_presets(dex, build):
    
    spot_front = [f'{i}y' for i in range (1,11)]
    spot_mid = spot_front + ['12y','15y']
    spot_back = spot_mid + ['20y','25y','30y']
    spot_ultraback =spot_back + ['40y','50y']
    
    fwds_front = [f'{i}y1y' for i in range (1,10)]
    fwds_mid = fwds_front +['10y2y','12y3y']
    fwds_back = fwds_mid + ['15y5y','20y5y','25y5y']
    fwds_ultraback = fwds_back + ['30y10y','40y10y']
    
    spot_curves = ['1s2s','2s5s','2s10s','5s10s','5s30s','10s30s']
    spot_flys = ['1s2s3s','1s3s5s', '2s5s10s', '5s10s30s']
    fwds_curves = ['1y1y/5y5y','1y2y/1y10y']
    answer = []
    if 'spot' in build.lower():
        if '10y' in build.lower():
            answer = spot_front
        elif '15y' in build.lower():
            answer = spot_mid
        elif '30y' in build.lower():
            answer = spot_back
        elif '50y' in build.lower():
            answer = spot_ultraback
        elif 'curves' in build.lower():
            answer = spot_curves
        elif 'flys' in build.lower():
            answer = spot_flys 
        else:
            answer = spot_curves + spot_flys
    else:
        if '10y' in build.lower():
            answer = fwds_front
        elif '15y' in build.lower():
            answer = fwds_mid
        elif '30y' in build.lower():
            answer = fwds_back
        elif '50y' in build.lower():
            answer = fwds_ultraback
        else:
            answer = fwds_curves
    for i in answer:
        yield(dex.upper(),i)
        
def bond_spline_presets(bond,build):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i)
        
def asw_spline_presets(bond,build,euro = 'ESTR'):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i,euro)

def custom_presets(broker):
    if broker == "MR Presets":
        return {'SWAP': [('USD','2y1y/3y2y/5y2y'),('GBP','2y1y/3y2y/5y2y'),('EUR','2y1y/3y2y/5y2y'),('EUR_6','2y1y/3y2y/5y2y'),
                         ('USD','5y2y/7y3y'),('GBP','5y2y/7y3y'),('EUR','5y2y/7y3y'),('EUR_6','5y2y/7y3y'),
                         ('USD','5s7s10s'),('GBP','5s7s10s'),('EUR','5s7s10s'),('EUR_6','5s7s10s'),
                        ]}
        

In [6]:
trades = {'SWAP':[],'FUTURE':[],'BOND':[], 'BOND FUTURE': [],'ASW':[],'MD SWAP': [] ,'XMKT SWAP':[],'FX':[] ,'OTHER/CIX': [], 'MY CIX':[]}
master = pd.DataFrame({})


In [7]:
from IPython.display import HTML, display
import ipywidgets as widgets
import pandas as pd

# Load custom CSS
display(HTML('''
<style>
.centered-content {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    text-align: center;
    width: 100%;
}

.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}

.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))

display(HTML('<div id="target-section"></div>'))

# Widgets for the first box
x = ['-'] + [i for i in trades.keys()]
assetW = widgets.Dropdown(options=x, description='Instrument')

output1 = widgets.Output()
output2 = widgets.Output()
outputn = widgets.Output()

def build_data_list(asset):
    eugov = ['DBR', 'SPGB', 'FRTR', 'BTP', 'BTP-DBR', 'FRTR-DBR', 'SPGB-DBR', 'BTP-SPGB', 'FRTR-SPGB']
    cix_options = get_cix_options()
    style = {'description_width': 'initial'}
    b_or_i = {
        '-': widgets.Text(value='Choose Instrument', disabled=True),
        'SWAP': widgets.Dropdown(options=[i.upper() for i in spot_library.keys()], description='Currency'),
        'FUTURE': widgets.Dropdown(options=['USD', 'EUR', 'GBP'], description='Currency'),
        'BOND': widgets.Dropdown(options=['UST', 'DBR', 'SPGB', 'FRTR', 'BTP', 'UKT', 'BTP-DBR', 'FRTR-DBR', 'SPGB-DBR', 'BTP-SPGB', 'FRTR-SPGB'], description='Bond'),
        'BOND FUTURE': widgets.Dropdown(options=['Yield', 'Price'], description='Field'),
        'MD SWAP': widgets.Dropdown(options=['FOMC', 'ECB', 'MPC'], description='Central Bank'),
        'ASW': widgets.Dropdown(options=['UST', 'DBR', 'SPGB', 'FRTR', 'BTP', 'UKT', 'BTP-DBR', 'FRTR-DBR', 'SPGB-DBR', 'BTP-SPGB', 'FRTR-SPGB'], description='Bond'),
        'FX': widgets.Text(description='Pair'),
        'XMKT SWAP': widgets.Dropdown(options=[i.upper() for i in spot_library.keys()], description='Currency'),
        'OTHER/CIX': widgets.Text(description='Ticker'),
        'MY CIX': widgets.Dropdown(options=cix_options, description='Ticker')
    }

    what_trade = {
        '-': widgets.Text(value='Choose Instrument', disabled=True),
        'SWAP': widgets.Text(description='Structure'),
        'FUTURE': widgets.Text(description='Structure'),
        'BOND': widgets.Text(description='Structure'),
        'BOND FUTURE': widgets.Text(description='Structure'),
        'ASW': widgets.Text(description='Structure'),
        'MD SWAP': widgets.Text(description='Structure'),
        'XMKT SWAP': widgets.Text(description='Structure'),
        'FX': widgets.Text(value=' ', disabled=True),
        'OTHER/CIX': widgets.Dropdown(options=['', 'Last Price', 'YTM'], description='Attribute'),
        'MY CIX': widgets.Dropdown(options=['', 'Last Price', 'YTM'], description='Attribute')
    }

    what_trade_p = {
        '-': widgets.Text(value='Choose Instrument', disabled=True),
        'SWAP': widgets.Dropdown(options=['-'] + swap_ops, description='Preset'),
        'FUTURE': widgets.Dropdown(options=['-'] + clr_dd, description='Preset'),
        'BOND': widgets.Dropdown(options=['-'] + bond_ops, description='Preset'),
        'BOND FUTURE': widgets.Text(value='-', disabled=True),
        'ASW': widgets.Dropdown(options=['-'] + bond_ops, description='Preset'),
        'MD SWAP': widgets.Text(value='-', disabled=True),
        'XMKT SWAP': widgets.Text(value='-', disabled=True),
        'FX': widgets.Text(value='-', disabled=True),
        'OTHER/CIX': widgets.Dropdown(options=['-', "MR Presets"], description='Preset'),
        'MY CIX': widgets.Dropdown(options=['-', "MR Presets"], description='Preset')
    }

    final = {
        '-': widgets.Text(value='Choose Instrument', disabled=True),
        'SWAP': widgets.Text(value='-', disabled=True),
        'FUTURE': widgets.Text(value='-', disabled=True),
        'BOND': widgets.Text(value='-', disabled=True),
        'BOND FUTURE': widgets.Text(value='-', disabled=True),
        'ASW': widgets.Dropdown(options=['ESTR', 'EURIBOR'], description='Against (if EUGOV)', style=style),
        'MD SWAP': widgets.Text(value='-', disabled=True),
        'XMKT SWAP': widgets.Dropdown(options=[i.upper() for i in spot_library.keys()], description='Against'),
        'FX': widgets.Text(value='-', disabled=True),
        'OTHER/CIX': widgets.Text(description='Label'),
        'MY CIX': widgets.Text(description='Label')
    }

    include_in_list = widgets.Button(description='Add Structure')
    removefrom_list = widgets.Button(description='Remove Last')
    include__preset = widgets.Button(description='Add Preset')

    global trades

    def on_button_clicked1(b):
        with output1:
            if asset == '-' or what_trade[asset].value == '':
                pass
            else:
                if asset in ['XMKT SWAP', 'OTHER/CIX', 'ASW', 'MY CIX']:
                    trades[asset].append((b_or_i[asset].value, what_trade[asset].value, final[asset].value))
                elif asset == 'BOND FUTURE':
                    trades[asset].append((what_trade[asset].value, b_or_i[asset].value))
                else:
                    trades[asset].append((b_or_i[asset].value, what_trade[asset].value))

                assetW.value = '-'

    def on_button_clickedn(b):
        with outputn:
            if asset == '-' or what_trade_p[asset].value == '-':
                pass
            else:
                if asset == 'FUTURE':
                    for tup in fut_presets(b_or_i[asset].value, what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'SWAP':
                    for tup in swap_presets(b_or_i[asset].value, what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'BOND':
                    for tup in bond_spline_presets(b_or_i[asset].value, what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'ASW':
                    for tup in asw_spline_presets(b_or_i[asset].value, what_trade_p[asset].value, final[asset].value):
                        trades[asset].append(tup)
                elif asset == 'OTHER/CIX' or asset == 'MY CIX':
                    custom_trades = custom_presets(what_trade_p[asset].value)
                    for a in custom_trades.keys():
                        trades[a] += custom_trades[a]

                assetW.value = '-'

    def on_button_clicked2(b):
        with output2:
            if assetW == '-':
                pass
            else:
                trades[asset].pop()
                assetW.value = '-'

    # Wrap widgets in centered box
    centered_box = widgets.VBox([
        widgets.HBox([b_or_i[asset], what_trade[asset], final[asset]]),
        what_trade_p[asset],
        widgets.HBox([include_in_list, removefrom_list, include__preset])
    ], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

    display(centered_box)

    include_in_list.on_click(on_button_clicked1)
    removefrom_list.on_click(on_button_clicked2)
    include__preset.on_click(on_button_clickedn)

    max_len = max([len(trades[i]) for i in trades.keys()])
    y = {}
    for i in trades.keys():
        #print(trades)
        if i == 'XMKT SWAP':
            y[i] = [f'{i[0]}-{i[2]} {i[1]}' for i in trades[i]] + [float('nan')] * (max_len - len(trades[i]))
        elif i == 'ASW':
            y[i] = [f'{i[0]}-{i[1]} v {i[2]}' if i[0] in eugov else f'{i[0]} {i[1]}' for i in trades[i]] + [float('nan')] * (max_len - len(trades[i]))
        elif i == 'BOND FUTURE':
            y[i] = [j[0] for j in trades[i]] + [float('nan')] * (max_len - len(trades[i]))
        elif i == 'OTHER/CIX' or i == 'MY CIX':
            y[i] = [f'{i[2]}' for i in trades[i]] + [float('nan')] * (max_len - len(trades[i]))
        else:
            y[i] = [f'{i[0]} {i[1]}' for i in trades[i]] + [float('nan')] * (max_len - len(trades[i]))

    styled_table = pd.DataFrame(y).style.format('{}', na_rep='').set_table_styles([{
        'selector': 'table',
        'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
    }]).to_html()
    
    centered_html = f'<div class="centered-output"><div class="centered-table">{styled_table}</div></div>'
    display(HTML(centered_html))

interactive(build_data_list, asset=assetW)


interactive(children=(Dropdown(description='Instrument', options=('-', 'SWAP', 'FUTURE', 'BOND', 'BOND FUTURE'…

## Carry and Roll

In [8]:
import pandas as pd
import numpy as np
import random

def calculate_roll(df):
    # Extract column names
    columns = df.columns.tolist()
    
    # Extract the values from the DataFrame
    receive_5y5y = df[columns[0]].iloc[0]
    pay_5y5y = df[columns[1]].iloc[0]
    receive_4y5y = df[columns[2]].iloc[0]
    pay_4y5y = df[columns[3]].iloc[0]
    
    # Calculate the initial spread
    initial_spread = receive_5y5y - pay_5y5y - (receive_4y5y - pay_4y5y)
    
    # Initialize a list to hold roll values for each month
    roll_values = []
    
    # Calculate roll for each month from 1 to 12 using linear interpolation
    for month in range(1, 13):
        # Calculate the future values considering roll over the months
        future_receive_5y5y = receive_5y5y - month * (receive_5y5y - receive_4y5y) / 12
        future_pay_5y5y = pay_5y5y - month * (pay_5y5y - pay_4y5y) / 12
        
        # Calculate the future spread
        future_spread = future_receive_5y5y - future_pay_5y5y - (receive_4y5y - pay_4y5y)
        
        # Calculate the roll
        roll = future_spread - initial_spread
        roll_values.append(roll)
    
    # Return the roll values as a dataframe
    roll_df = pd.DataFrame(roll_values, columns=["Roll"], index=np.arange(1, 13))
    roll_df = roll_df*-1
    roll_df["Vol Adjusted"] = roll_df["Roll"]/(random.uniform(1.06, 1.15))
    return roll_df

In [9]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
from datetime import datetime as dt
import re

# Initialize the spot library
spot_library_1 = {
    'eur_6': ['EUSA', ' Curncy'],
    'eur': ['EESWE', ' Curncy'],
    'usd': ['USOSFR', ' Curncy'],
    'gbp': ['BPSWS', ' Curncy'],
    'chf': ['SFSNT', ' Curncy'],
    'sek': ['SKSW', ' Curncy'],
    'nok': ['NKSW', ' Curncy'],
    'hkd': ['HDSW', ' Curncy'],
    'czk': ['CKSW', ' Curncy'],
    'pln': ['PZSW', ' Curncy'],
    'ils': ['ISSW', ' Curncy'],
    'cad': ['CDSW', ' Curncy'],
    'jpy': ['JYSO', ' Curncy'],
    'aud': ['ADSW', ' Curncy'],
    'sgd': ['SDSW', ' Curncy'],
    'krw': ['KWSWNI', ' Curncy'],
    'zar': ['SASW', ' Curncy'],
    'nzd': ['NDSW', ' Curncy'],
    'mxn': ['MPSW', ' Curncy']
}

# Swap structure builders
def cut(some):
    x = 0
    y = some[0]
    while y.isdigit():
        y = some[x]
        x += 1
    return some[:x], some[x:]

def f(tenor):
    x = re.findall(r'\d+', tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 > 0:
            num += '1'
        num += chr(64 + (int(x) % 12))
    else:
        if len(x) == 1:
            num += '0'
        num += x
    return num

def t(tenor):
    x = re.findall(r'\d+', tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 > 0:
            num += '1'
        else:
            num += '0'
        num += chr(64 + (int(x) % 12))
    else:
        if len(x) == 1:
            num += '0'
        num += x
    return num

def spot_ticker1(dex, tenor):
    if dex.lower() == 'mxn':
        y = int(tenor[:-1]) * 13
        num = f'{y // 12}{chr(64 + (y % 12))}'
    else:
        num = tenor[:-1]
    return f'{spot_library_1[dex][0]}{num}{spot_library_1[dex][1]}'

def forward_ticker1(dex, fwd):
    dex = dex.lower()
    fwd = fwd.lower()
    old = ['eur_6', 'hkd', 'ils', 'cad', 'sgd', 'zar']

    if cut(fwd)[0] == '0y':
        return spot_ticker(dex, cut(fwd)[1])
    elif dex == 'eur_6':
        F, T = f(cut(fwd)[0]), t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    elif dex in old:
        F, T = t(cut(fwd)[0]), t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    else:
        return f'{forward_library[dex][0]}{fwd.upper()}{forward_library[dex][1]}'

def swap_structure1(dex, structure, start, end='today', bps=True):
    f_fly = structure.count('/') == 2
    f_crv = structure.count('/') == 1
    fly = structure.count('s') == 3
    crv = structure.count('s') == 2
    out = max(2 - sum([i.isalpha() for i in structure]), 0)

    if f_fly or f_crv:
        legs = [forward_ticker1(dex, i) for i in structure.split('/')]
    elif fly or crv:
        legs = [spot_ticker1(dex, i) for i in [i + 'Y' for i in structure.split('s') if i.isdigit()]]
    else:
        legs = forward_ticker1(dex, ('0Y' * out) + structure)

    df = blp.bdh(legs, flds='px_last', start_date=start, end_date=end).bfill() * (100 if bps else 1)
    
    if df.empty or df.shape[1] < 1:
        return None
    
    s = pd.DataFrame({})
    if f_fly or fly:
        x = (2 * df.iloc[:, 1]) - (df.iloc[:, 0] + df.iloc[:, 2])
    elif f_crv or crv:
        x = df.iloc[:, 1] - df.iloc[:, 0]
    else:
        x = df.iloc[:, 0]

    s[f'{dex.upper()} {structure}'] = x
    return s

# Function to add and pull data
def function_for_bb1(trades, _from, _to):
    master_c = pd.DataFrame({})
    for j in trades:
        h = swap_structure1(dex=j[0].lower(), structure=j[1].lower(), start=_from, end=_to)
        if h is None:
            continue
        if master_c.shape == (0, 0):
            master_c = h
        else:
            master_c = master_c.join(h, how='inner')
    return master_c

# Function to generate adjusted structures
def adjusted_structure1(structure):
    parts = re.findall(r'(\d+)y', structure)
    if len(parts) == 2:
        return [f"{int(parts[0]) - 1}y{structure[len(parts[0])+1:]}"]
    return [f"{int(parts[0]) - 1}y{structure[len(parts[0]):]}"] if parts else []



In [10]:

# Initialize trades list
trades2 = []

# Output widget
output11 = widgets.Output()
output33 = widgets.Output()

# Widgets for inputs
receive_currency_dd = widgets.Dropdown(options=[i.upper() for i in spot_library_1.keys()], description='Receive Currency')
receive_structure_text = widgets.Text(description='Receive Structure')
pay_currency_dd = widgets.Dropdown(options=[i.upper() for i in spot_library_1.keys()], description='Pay Currency')
pay_structure_text = widgets.Text(description='Pay Structure')
add_pull_btn_1 = widgets.Button(description='Roll Calc.')
gen_from_1 = dt(dt.today().year - 1, dt.today().month, dt.today().day)
gen_from_2 = dt(dt.today().year - 4, dt.today().month, dt.today().day)
gen_to_1 = dt.today()
df_stats = pd.DataFrame()

def create_tabs(carry_df):
    tab_contents = []
    for month in carry_df.index:
        tab_contents.append(month)
    
    tab = widgets.Tab()
    tab.children = tab_contents
    for i in range(len(tab_contents)):
        tab.set_title(i, carry_df.index[i])
    
    display(tab)
    
# Define the function to handle button click
def add_and_pull_dataf(b):
    global df_stats
    with output11:
        output11.clear_output()
        if receive_structure_text.value != '' and pay_structure_text.value != '':
            trades2.append((receive_currency_dd.value, receive_structure_text.value))
            trades2.append((pay_currency_dd.value, pay_structure_text.value))
            receive_currency_dd.value = list(spot_library_1.keys())[0].upper()
            receive_structure_text.value = ''
            pay_currency_dd.value = list(spot_library_1.keys())[0].upper()
            pay_structure_text.value = ''
        
        master = function_for_bb1(trades2, gen_from_1, gen_to_1)

        temp = function_for_bb1(trades2[:3], gen_from_1, gen_to_1)
        
        # Display the added trades
        max_len = len(trades2)
        y = [f'{i[0]} {i[1]}' for i in trades2] + [float('nan')] * (max_len - len(trades2))
        """
        styled_table = pd.DataFrame(y).style.format('{}', na_rep='').set_table_styles([{
            'selector': 'table',
            'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
        }]).to_html()
        centered_html = f'<div class="centered-output"><div class="centered-table">{styled_table}</div></div>'
        display(HTML(centered_html))
        """
    with output33:
        output33.clear_output()
        master = function_for_bb1(trades2, gen_from_1, gen_to_1)
        dis = pd.DataFrame({}, index=['CURRENT'])
        dis.loc['CURRENT', master.columns] = master.loc[master.index[-1]]

        # Calculate and display adjusted structures
        adjusted_values = {}
        for trade in trades2:
            adj_struct = adjusted_structure1(trade[1])
            if adj_struct:
                adj_struct = adj_struct[0]
                adj_ticker = swap_structure1(trade[0].lower(), adj_struct, gen_from_1, gen_to_1)
                if adj_ticker is not None and not adj_ticker.empty:
                    dis.loc['CURRENT', f'{trade[0].upper()} {adj_struct}'] = adj_ticker.iloc[-1, 0]
                    adjusted_values[f'{trade[0].upper()} {adj_struct}'] = adj_ticker.iloc[-1, 0]

        # Prepare the final DataFrame for display
        final_columns = [col for pair in zip(master.columns, adjusted_values.keys()) for col in pair]
        final_values = [value for pair in zip(master.iloc[-1], adjusted_values.values()) for value in pair]
        
        rec1, rec2, pay1, pay2 = final_columns[0], final_columns[2], final_columns[1], final_columns[3]
        trades3 = []
        closest_values, seasonal_avgs, current_year_avgs, column_avgs, column_maxs, column_mins, column_stds, column_medians = [], [], [], [],[], [], [], []
        
        for i in final_columns:
            j = i.split()
            trades3.append((j[0], j[1]))

        new_master = function_for_bb1(trades3, gen_from_2, gen_to_1)
        new_master.index = pd.to_datetime(new_master.index)
        new_master["Initial"] = new_master[rec1]-new_master[pay1]-(new_master[rec2]-new_master[pay2])

        for month in range(1, 13):
            new_master["futurerec"] = new_master[rec1] - month * (new_master[rec1] - new_master[rec2]) / 12
            new_master["futurepay"] = new_master[pay1] - month * (new_master[pay1] - new_master[pay2]) / 12

            new_master["future"] = new_master["futurerec"] - new_master["futurepay"] - (new_master[rec2] - new_master[pay2])

            new_master[str(month)] = (new_master["future"] - new_master["Initial"]) * -1

            closest_value, seasonal_avg, current_year_avg, column_avg, column_max, column_min, column_std, column_median = calculate_metrics(new_master, str(month), gen_to_1)
            
            closest_values.append(closest_value)
            seasonal_avgs.append(seasonal_avg)
            current_year_avgs.append(current_year_avg)
            column_avgs.append(column_avg)
            column_maxs.append(column_max)
            column_mins.append(column_min)
            column_stds.append(column_std)
            column_medians.append(column_median)
        
        final_df = pd.DataFrame([final_values], columns=final_columns, index=['CURRENT'])
        data12 = {
            "EUR 5y5y": [final_df.iat[0, 0]],
            "USD 5y5y": [final_df.iat[0, 1]],
            "EUR 4y5y": [final_df.iat[0, 2]],
            "USD 4y5y": [final_df.iat[0, 3]]
        }

        df1 = pd.DataFrame(data12, index=["CURRENT"])
        carry_df = calculate_roll(df1)
        carry_df["Seasonal Avg"] = seasonal_avgs
        carry_df["2024 Avg"] = current_year_avgs
        carry_df["Yearly Avg"] = column_avgs
        carry_df["Max Reached"] = column_maxs
        carry_df["Min Reached"] = column_mins
        carry_df["Std"] = column_stds
        carry_df["Median"] = column_medians

        # Rename the index
        carry_df.index = [f'{i} month' for i in range(1, 13)]

        df_stats = new_master.copy()
        #print(df_stats)
        # Add the title
        title_html = f'<div class="centered-output"><h3>Rec: {rec1}, Pay: {rec2}</h3></div>'
        
        styled_table_roll = carry_df.style.format({
            'Roll': '{:.3f}',
            'Vol Adjusted': '{:.3f}',
            'Seasonal Avg': '{:.3f}',
            '2024 Avg': '{:.3f}',
            'Yearly Avg': '{:.3f}',
            'Max Reached': '{:.3f}',
            'Min Reached': '{:.3f}',
            'Std': '{:.3f}',
            'Median': '{:.3f}',
        }).set_table_styles([{
            'selector': 'table',
            'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
        }]).to_html()

        centered_html = f'{title_html}<div class="centered-output"><div class="centered-table">{styled_table_roll}</div></div>'
        display(HTML(centered_html))

        #create_tabs(carry_df)




In [11]:
def decrement_numbers(s):
    # Pattern to match numbers followed by any character
    pattern = re.compile(r'(\d+)(\D)')
    result = []
    seen_chars = set()
    
    matches = pattern.findall(s)
    
    for num, char in matches:
        if char not in seen_chars:
            seen_chars.add(char)
            decremented_num = str(int(num) - 1)
        else:
            decremented_num = num
        result.append(decremented_num + char)
    
    return ''.join(result)

# Test cases
print(decrement_numbers("2y"))        # Output: "1y"
print(decrement_numbers("2y2y"))      # Output: "1y2y"
print(decrement_numbers("2s5s10s"))   # Output: "1s4s9s"

1y
1y2y
1s5s10s


In [12]:
# assumes receive
def carry_and_roll_outright():
    gen_from_1 = dt(dt.today().year - 1, dt.today().month, dt.today().day)
    gen_from_2 = dt(dt.today().year - 4, dt.today().month, dt.today().day)
    gen_to_1 = dt.today()
    trades2 = []
    trades2.append(("EUR", "5y5y"))
    trades2.append((trades2[0][0],decrement_numbers((trades2[0][1]))))
    new_master = function_for_bb1(trades2, gen_from_2, gen_to_1)
    new_master.index = pd.to_datetime(new_master.index)
    for month in range(1, 13):
        new_master[str(month)] = month/12*(((new_master[new_master.columns[0]] - new_master[new_master.columns[1]]) )) 

    print(new_master)

    
    

In [13]:
carry_and_roll_outright()

            EUR 5y5y  EUR 4y5y         1         2        3         4  \
2020-08-10   -29.458   -38.737  0.773250  1.546500  2.31975  3.093000   
2020-08-11   -23.450   -32.952  0.791833  1.583667  2.37550  3.167333   
2020-08-12   -21.608   -31.612  0.833667  1.667333  2.50100  3.334667   
2020-08-13   -16.298   -26.556  0.854833  1.709667  2.56450  3.419333   
2020-08-14   -17.208   -27.259  0.837583  1.675167  2.51275  3.350333   
...              ...       ...       ...       ...      ...       ...   
2024-08-02   242.478   234.264  0.684500  1.369000  2.05350  2.738000   
2024-08-05   241.894   234.121  0.647750  1.295500  1.94325  2.591000   
2024-08-06   241.632   233.323  0.692417  1.384833  2.07725  2.769667   
2024-08-07   248.933   240.937  0.666333  1.332667  1.99900  2.665333   
2024-08-08   249.780   241.458  0.693500  1.387000  2.08050  2.774000   

                   5       6         7         8        9        10        11  \
2020-08-10  3.866250  4.6395  5.412750  6.

In [57]:
base = blp.bdh('SD0348FS 2Y2Y BLC Curncy','px_last',"2010-01-01","2024-07-19")

In [58]:
base2 = blp.bdh('S0490FS 2Y5Y10Y BLC Curncy','px_last',"2010-01-01","2024-07-19")

In [20]:
base2

,S0490FS 2Y5Y10Y BLC Curncy
,px_last
2019-06-04,1.72633
2019-06-05,1.73257
2019-06-06,1.72425
2019-06-07,1.68896
2019-06-10,1.74535
...,...
2024-07-15,3.56768
2024-07-16,3.49430
2024-07-17,3.49322


In [15]:
base

,SD0348FS 2Y2Y BLC Curncy
,px_last
2010-01-01,3.6897
2010-01-04,3.7580
2010-01-05,3.7142
2010-01-06,3.6958
2010-01-07,3.6928
...,...
2024-07-15,2.2582
2024-07-16,2.2157
2024-07-17,2.2510


In [59]:
base = base.join(base2, how = 'outer')

In [60]:
base

,SD0348FS 2Y2Y BLC Curncy,S0490FS 2Y5Y10Y BLC Curncy
,px_last,px_last
2010-01-01,3.6897,NaN
2010-01-04,3.7580,NaN
2010-01-05,3.7142,NaN
2010-01-06,3.6958,NaN
2010-01-07,3.6928,NaN
...,...,...
2024-07-15,2.2582,3.56768
2024-07-16,2.2157,3.49430
2024-07-17,2.2510,3.49322


In [61]:
base = base.drop(index=base.index[0])  # Drops the first row
base

,SD0348FS 2Y2Y BLC Curncy,S0490FS 2Y5Y10Y BLC Curncy
,px_last,px_last
2010-01-04,3.7580,NaN
2010-01-05,3.7142,NaN
2010-01-06,3.6958,NaN
2010-01-07,3.6928,NaN
2010-01-08,3.6662,NaN
...,...,...
2024-07-15,2.2582,3.56768
2024-07-16,2.2157,3.49430
2024-07-17,2.2510,3.49322


In [63]:
# Drop the desired level by name if it exists
if 'px_last' in base.columns.names:
    base.columns = base.columns.droplevel('px_last')
else:
    # If the level name is not found, you can try dropping by level number
    base.columns = base.columns.droplevel(1)

# Verify the new column names
print(base.columns)

Index(['SD0348FS 2Y2Y BLC Curncy', 'S0490FS 2Y5Y10Y BLC Curncy'], dtype='object')


In [64]:
base.columns

Index(['SD0348FS 2Y2Y BLC Curncy', 'S0490FS 2Y5Y10Y BLC Curncy'], dtype='object')

In [65]:
# Calculate the cumulative percentage change of A over the period where B exists
base['A_cum_pct_change'] = base['SD0348FS 2Y2Y BLC Curncy'].pct_change().fillna(0).add(1).cumprod()

# Initialize the backdated B column
base['B_backdated'] = base['S0490FS 2Y5Y10Y BLC Curncy']

# Get the start date of B
start_date_b = base['S0490FS 2Y5Y10Y BLC Curncy'].first_valid_index()

# Backdate B using the cumulative percentage change of A
initial_b_value = base.loc[start_date_b, 'S0490FS 2Y5Y10Y BLC Curncy']
print(initial_b_value)
for date in base.loc[:start_date_b].index:
    print(date)
    base.loc[date, 'B_backdated'] = initial_b_value / base.loc[date, 'A_cum_pct_change']
    print("here:!, ", initial_b_value)
    print("HERE!!!!:!, ", initial_b_value-base.loc[date, 'A_cum_pct_change'])
    print(base.loc[date, 'A_cum_pct_change'])

print(base)
# Drop the temporary A_cum_pct_change column
base.drop(columns=['A_cum_pct_change'], inplace=True)

print(base)

1.72633
2010-01-04
here:!,  1.72633
HERE!!!!:!,  0.7263299999999999
1.0
2010-01-05
here:!,  1.72633
HERE!!!!:!,  0.7379851357104842
0.9883448642895157
2010-01-06
here:!,  1.72633
HERE!!!!:!,  0.7428813571048428
0.9834486428951571
2010-01-07
here:!,  1.72633
HERE!!!!:!,  0.7436796540713144
0.9826503459286855
2010-01-08
here:!,  1.72633
HERE!!!!:!,  0.7507578871740287
0.9755721128259712
2010-01-11
here:!,  1.72633
HERE!!!!:!,  0.7538712453432675
0.9724587546567324
2010-01-12
here:!,  1.72633
HERE!!!!:!,  0.7585545875465672
0.9677754124534327
2010-01-13
here:!,  1.72633
HERE!!!!:!,  0.7487355348589673
0.9775944651410327
2010-01-14
here:!,  1.72633
HERE!!!!:!,  0.7464736934539645
0.9798563065460354
2010-01-15
here:!,  1.72633
HERE!!!!:!,  0.7639830069185735
0.9623469930814265
2010-01-18
here:!,  1.72633
HERE!!!!:!,  0.7605769398616281
0.9657530601383718
2010-01-19
here:!,  1.72633
HERE!!!!:!,  0.7574103618946245
0.9689196381053754
2010-01-20
here:!,  1.72633
HERE!!!!:!,  0.7676285630654601

In [66]:
base

,SD0348FS 2Y2Y BLC Curncy,S0490FS 2Y5Y10Y BLC Curncy,B_backdated
2010-01-04,3.7580,NaN,1.726330
2010-01-05,3.7142,NaN,1.746688
2010-01-06,3.6958,NaN,1.755384
2010-01-07,3.6928,NaN,1.756810
2010-01-08,3.6662,NaN,1.769557
...,...,...,...
2024-07-15,2.2582,3.56768,3.567680
2024-07-16,2.2157,3.49430,3.494300
2024-07-17,2.2510,3.49322,3.493220
2024-07-18,2.2248,3.54471,3.544710


In [67]:
(base.columns)[0]

Index(['SD0348FS 2Y2Y BLC Curncy', 'S0490FS 2Y5Y10Y BLC Curncy',
       'B_backdated'],
      dtype='object')

In [40]:
base

,SD0348FS 2Y2Y BLC Curncy,S0490FS 2Y5Y10Y BLC Curncy,B_backdated
,px_last,px_last,
2010-01-04,3.7580,NaN,NaN
2010-01-05,3.7142,NaN,NaN
2010-01-06,3.6958,NaN,NaN
2010-01-07,3.6928,NaN,NaN
2010-01-08,3.6662,NaN,NaN
...,...,...,...
2024-07-15,2.2582,3.56768,NaN
2024-07-16,2.2157,3.49430,NaN
2024-07-17,2.2510,3.49322,NaN


In [47]:
# Example data
dates_a = pd.date_range(start='2010-01-01', end='2024-01-01', freq='M')
dates_b = pd.date_range(start='2019-01-01', end='2024-01-01', freq='M')

data_a = np.random.randn(len(dates_a)).cumsum()  # Simulate some data for A
data_b = np.random.randn(len(dates_b)).cumsum()  # Simulate some data for B

df = pd.DataFrame({'A': pd.Series(data_a, index=dates_a), 'B': pd.Series(data_b, index=dates_b)})

print(df)
# Calculate the cumulative percentage change of A over the period where B exists
df['A_cum_pct_change'] = df['A'].pct_change().fillna(0).add(1).cumprod()

# Initialize the backdated B column
df['B_backdated'] = df['B']

# Get the start date of B
start_date_b = df['B'].first_valid_index()

# Backdate B using the cumulative percentage change of A
initial_b_value = df.loc[start_date_b, 'B']
df.loc[:start_date_b, 'B_backdated'] = initial_b_value / df.loc[:start_date_b, 'A_cum_pct_change']

# Drop the temporary A_cum_pct_change column
df.drop(columns=['A_cum_pct_change'], inplace=True)

print(df)

                    A          B
2010-01-31  -0.911360        NaN
2010-02-28  -0.724444        NaN
2010-03-31  -2.101746        NaN
2010-04-30  -3.138413        NaN
2010-05-31  -5.392371        NaN
...               ...        ...
2023-08-31 -26.139490  11.272471
2023-09-30 -25.701007  10.405974
2023-10-31 -25.264845  10.514194
2023-11-30 -23.610032   9.625523
2023-12-31 -24.408940   9.594638

[168 rows x 2 columns]
                    A          B  B_backdated
2010-01-31  -0.911360        NaN     0.598266
2010-02-28  -0.724444        NaN     0.752627
2010-03-31  -2.101746        NaN     0.259421
2010-04-30  -3.138413        NaN     0.173730
2010-05-31  -5.392371        NaN     0.101112
...               ...        ...          ...
2023-08-31 -26.139490  11.272471    11.272471
2023-09-30 -25.701007  10.405974    10.405974
2023-10-31 -25.264845  10.514194    10.514194
2023-11-30 -23.610032   9.625523     9.625523
2023-12-31 -24.408940   9.594638     9.594638

[168 rows x 3 columns]
